In [1]:
# conda env create -f llama.yml

In [1]:
import gc
import os
import json
import torch
torch.cuda.is_initialized()


False

In [13]:
# personal HF token
HF_TOKEN = "YOUR HF TOKEN"


In [3]:
# DeepSpeed Config file
with open('/home/hyeonjj/hongseoi/LLM_fine_tuning_llama3_8b/ds_config_zero3.json') as f:
    ds_config = json.load(f)
ds_config_dict=dict(zero3=ds_config)

In [4]:
ds_config_dict["zero3"]

{'fp16': {'enabled': 'auto',
  'loss_scale': 0,
  'loss_scale_window': 1000,
  'initial_scale_power': 16,
  'hysteresis': 2,
  'min_loss_scale': 1},
 'bf16': {'enabled': 'auto'},
 'optimizer': {'type': 'AdamW',
  'params': {'lr': 'auto',
   'weight_decay': 'auto',
   'torch_adam': True,
   'adam_w_mode': True}},
 'scheduler': {'type': 'WarmupLR',
  'params': {'warmup_min_lr': 'auto',
   'warmup_max_lr': 'auto',
   'warmup_num_steps': 'auto'}},
 'zero_optimization': {'stage': 3,
  'offload_optimizer': {'device': 'cpu', 'pin_memory': True},
  'offload_param': {'device': 'cpu', 'pin_memory': True},
  'overlap_comm': True,
  'contiguous_gradients': True,
  'sub_group_size': 1000000000.0,
  'reduce_bucket_size': 'auto',
  'stage3_prefetch_bucket_size': 'auto',
  'stage3_param_persistence_threshold': 'auto',
  'stage3_max_live_parameters': 1000000000.0,
  'stage3_max_reuse_distance': 1000000000.0,
  'stage3_gather_16bit_weights_on_model_save': 'auto'},
 'gradient_accumulation_steps': 1,
 'gr

In [5]:
# import os
# from accelerate.utils import write_basic_config

# write_basic_config()  # Write a config file
# os._exit(00)  # Restart the notebook
torch.cuda.is_initialized()


False

In [7]:
# import torch
# DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# print(f"Device: {DEVICE}")
# print(f"CUDA Version: {torch.version.cuda}")
# print(f"Pytorch {torch.__version__}")

# # Check the type and quantity of GPUs
# if torch.cuda.is_available():
#     print('Num CPUs:', os.cpu_count())
#     print('Num GPUs:', torch.cuda.device_count())
#     print('GPU Type:', torch.cuda.get_device_name(0))
# torch.cuda.is_initialized()



### Llama_3_8b

In [6]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B"

In [7]:
# dataset setting
from datasets import load_dataset
from transformers import AutoTokenizer

dataset_name = "mlabonne/orpo-dpo-mix-40k"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42).select(range(100)) # Only use 1000 samples for quick demo

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def format_chat_template(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)
dataset = dataset.train_test_split(test_size=0.01)

torch.cuda.is_initialized()


/home/hyeonjj/.conda/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [8]:
from accelerate import notebook_launcher
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)


In [9]:

def main():
    
    from transformers import BitsAndBytesConfig
    from trl import ORPOConfig, ORPOTrainer, setup_chat_format
    from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
    from accelerate import Accelerator

#     from accelerate import FullyShardedDataParallelPlugin
#     from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

#     fsdp_plugin = FullyShardedDataParallelPlugin(
#         state_dict_config=FullStateDictConfig(offload_to_cpu=False, rank0_only=False),
#         optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=False, rank0_only=False),
#     )

#     accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

    from accelerate import Accelerator, DeepSpeedPlugin
    deepspeed_plugin = DeepSpeedPlugin(
            hf_ds_config=ds_config_dict["zero3"],
            gradient_accumulation_steps=4,
            gradient_clipping=1.0,
            zero_stage=3,
            offload_optimizer_device=None,
            offload_param_device=None,
            zero3_save_16bit_model=True,
            zero3_init_flag=True,
        )
    
    kwargs = {
        "fp16.enabled": True, 
        "fp16.auto_cast": False,
        "bf16.enabled": False,
        "optimizer.params.lr": 8e-6,
        "optimizer.params.weight_decay": 0.0,
        "scheduler.params.warmup_min_lr": 0.0,
        "scheduler.params.warmup_max_lr": 5e-5,
        "scheduler.params.warmup_num_steps": 0,
        "train_micro_batch_size_per_gpu": 1,
        "gradient_clipping": 1.0,
        "train_batch_size": 1,
        "zero_optimization.reduce_bucket_size": 5e5,
        "zero_optimization.stage3_prefetch_bucket_size": 5e5,
        "zero_optimization.stage3_param_persistence_threshold": 5e5,
#         "zero_optimization.stage3_gather_16bit_weights_on_model_save": False,
    }
    deepspeed_plugin.deepspeed_config_process(**kwargs)
    
    accelerator = Accelerator(deepspeed_plugin=deepspeed_plugin, mixed_precision="fp16")
#     accelerator = Accelerator(mixed_precision='fp16')
#     accelerator = Accelerator()
    
    device_map = {"": accelerator.process_index}
#     device_map = {"": "cuda:" + str(int(os.environ.get("LOCAL_RANK") or 0))}
#     device_map={'':torch.cuda.current_device()}

    
    # QLoRA config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_storage=torch.float16,
    )

    # LoRA config
    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
#         target_modules=["all_linear"],
        target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
    )
    
    base_model = "meta-llama/Meta-Llama-3-8B"
    new_model = "Llama-3-8B_FT_ORPO_9k"
    
    tokenizer = AutoTokenizer.from_pretrained(base_model, token=HF_TOKEN)
    
#     tokenizer.pad_token = tokenizer.eos_token

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
#         device_map="auto",
#         device_map=device_map,
        token=HF_TOKEN,
        attn_implementation="eager",
        torch_dtype=torch.float16,
    )
    
    model, tokenizer = setup_chat_format(model, tokenizer)
    model = prepare_model_for_kbit_training(model)
    
    dataset_name = "mlabonne/orpo-dpo-mix-40k"
    dataset = load_dataset(dataset_name, split="all")
    dataset = dataset.shuffle(seed=42).select(range(900)) # Only use 30 samples for test

    def format_chat_template(row):
        row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
        row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
        return row

    dataset = dataset.map(
        format_chat_template,
        num_proc= os.cpu_count(),
    )
    dataset = dataset.train_test_split(test_size=0.01)
    
    torch.cuda.empty_cache()
    
    orpo_args = ORPOConfig(
        learning_rate=8e-6,
        lr_scheduler_type="linear",
        max_length=1024,
        max_prompt_length=512,
        beta=0.1,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        num_train_epochs=1,
        evaluation_strategy="steps",
#         eval_strategy="steps",
        eval_steps=0.2,
        logging_steps=1,
        warmup_steps=10,
        report_to="none",
        output_dir="./results/",
        remove_unused_columns=False,
#         fp16=True,
#         bf16=False,
#         fsdp="full_shard",
#         fsdp_config={'min_num_params': 2000, 'offload_params': False, 'sharding_strategy': 1},
        ddp_find_unused_parameters=False,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs = {"use_reentrant": True}, #must be false for DDP
    )

    trainer = ORPOTrainer(
        model=model,
        args=orpo_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        peft_config=peft_config,
        tokenizer=tokenizer,
    )

    print(device_map)
    print(f'n_gpu: {orpo_args.n_gpu}; Mode: {orpo_args.parallel_mode}')
    print(f'Num Processes: {accelerator.num_processes}; Device: {accelerator.device}; Process Index: {accelerator.process_index}')
    print(f'Accel Type: {accelerator.distributed_type}')

    
    trainer.train()
    trainer.save_model(new_model)
    
torch.cuda.is_initialized()

    

False

In [ ]:
! nvidia-smi

Thu May 23 04:17:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    Off | 00000000:1F:00.0 Off |                  Off |
| 30%   54C    P2             268W / 300W |  32959MiB / 49140MiB |     97%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
torch.cuda.is_initialized()


False

In [14]:
notebook_launcher(main, num_processes=2)

Launching training on 2 GPUs.
[2024-05-23 04:26:53,811] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-05-23 04:26:53,840] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Pl

/home/hyeonjj/.conda/envs/llama/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/hyeonjj/.conda/envs/llama/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2024-05-23 04:26:54,339] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-23 04:26:54,430] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-05-23 04:26:54,432] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.80s/it]
/home/hyeonjj/.conda/envs/llama/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/hyeonjj/.conda/envs/llama/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Map: 100%|██████████| 9/9 [00:00<00:00, 233.32 exampl

{'': 0}
n_gpu: 1; Mode: ParallelMode.DISTRIBUTED
Num Processes: 2; Device: cuda:0; Process Index: 0
Accel Type: DEEPSPEED
{'': 1}
n_gpu: 1; Mode: ParallelMode.DISTRIBUTED
Num Processes: 2; Device: cuda:1; Process Index: 1
Accel Type: DEEPSPEED
[2024-05-23 04:27:13,936] [WARNING] [engine.py:1188:_do_optimizer_sanity_check] **** You are using ZeRO with an untested optimizer, proceed with caution *****


Could not estimate the number of tokens of the input, floating-point operations will not be computed
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
23,1.998400,2.039481,9.786100,0.920000,0.511000,-0.089396,-0.110169,0.800000,0.020774,-1.101692,-0.893956,-1.127855,-1.001212,1.124311,-0.548817,0.324846
46,2.341400,1.685729,9.741700,0.924000,0.513000,-0.085441,-0.105359,0.800000,0.019918,-1.053587,-0.854405,-1.204275,-1.016903,1.081197,-0.550580,0.320067
69,1.749400,1.203926,9.826000,0.916000,0.509000,-0.080037,-0.098318,0.800000,0.018280,-0.983177,-0.800374,-1.230397,-1.031906,1.027389,-0.554661,0.309851
92,1.391600,1.173325,9.742200,0.924000,0.513000,-0.078704,-0.095225,0.800000,0.016521,-0.952250,-0.787044,-1.128900,-0.974851,1.010083,-0.563254,0.289948


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
23,1.998400,2.039481,9.784400,0.920000,0.511000,-0.107637,-0.104817,0.600000,-0.002820,-1.048172,-1.076372,-1.101501,-0.983448,2.560827,-0.719484,-0.001432
46,2.341400,1.685729,9.739900,0.924000,0.513000,-0.102144,-0.100671,0.400000,-0.001474,-1.006705,-1.021444,-1.122681,-1.050238,1.967415,-0.710285,0.014548
69,1.749400,1.203926,9.824600,0.916000,0.509000,-0.094392,-0.094296,0.400000,-0.000096,-0.942960,-0.943917,-1.117578,-1.097724,1.154178,-0.700229,0.030355
92,1.391600,1.173325,9.742200,0.924000,0.513000,-0.091693,-0.092047,0.400000,0.000354,-0.920469,-0.916926,-1.064152,-1.049017,1.115890,-0.696568,0.036390


/home/hyeonjj/.conda/envs/llama/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hyeonjj/.conda/envs/llama/lib/python3.9/site-packages/peft/utils/save_and_load.py:209: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


### Merge Adapter with Base model

In [17]:
# Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()


In [18]:
# Reload tokenizer and model

from trl import setup_chat_format

tokenizer = AutoTokenizer.from_pretrained(base_model, token=HF_TOKEN)
fp16_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN,
)
fp16_model, tokenizer = setup_chat_format(fp16_model, tokenizer)


[2024-05-23 05:00:53,715] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/hyeonjj/.conda/envs/llama/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


In [19]:
# merge fine tuned adapter
from peft import PeftModel

new_model = './Llama-3-8B_FT_ORPO_9k'

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, new_model)
model = model.merge_and_unload()

In [20]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

### Inference with Fine-tuned model

In [23]:
# fine tuned model inference
#%%time
DEVICE = "cuda"
question = 'What is the basic structure of a SQL query to join to tables on a field like ID'
# question = 'When is labor day celebrated in USA'

# Tokenize the prompt
inputs = tokenizer(question, return_tensors="pt").to(DEVICE)
# Generate the outputs from prompt
generate_ids = model.generate(**inputs, max_new_tokens=256)
# Decode the generated output
generated_text = tokenizer.batch_decode(generate_ids,
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)[0]

print('generated_text: ', generated_text)

generated_text:  What is the basic structure of a SQL query to join to tables on a field like ID?
For example, if I have a table of products and a table of orders, how would I write a query to get all of the products that were ordered?
It would be helpful to know your RDBMS. But the general syntax is this:
SELECT p.ProductID, p.ProductName, o.OrderID
FROM Products p
JOIN Orders o
ON p.ProductID = o.ProductID
The JOIN keyword can be used in a few different ways. The one you are looking for is the INNER JOIN. This is the default join type, so you don't need to specify it, but it is good practice to always be explicit.
The ON clause is used to specify the relationship between the two tables. In this case, the ProductID field from the Products table matches the ProductID field from the Orders table.
The INNER JOIN clause is used to retrieve all rows from both tables where the specified condition is true. In this case, the condition is that the ProductID field from the Products table matche

In [24]:
# base model result
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline("What is the basic structure of a SQL query to join to tables on a field like ID")


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'What is the basic structure of a SQL query to join to tables on a field like ID?\nWhat is the basic structure of a SQL query to join to tables on a field like ID?\nI am trying to join two tables on a field called ID. I am using SQL in MySQL. I am new to SQL. I have been looking at examples but I am not sure how to modify them to join on the ID field. Here is the basic structure of what I am trying to do:\nSELECT table1.ID, table1.name, table2.ID, table2.address\nFROM table1, table2\nWHERE table1.ID = table2.ID\nWhat is the basic structure of a SQL query to join to tables on a field like ID?\nI am trying to join two tables on a field called ID. I am using SQL in MySQL. I am new to SQL. I have been looking at examples but I am not sure how to modify them to join on the ID field. Here is the basic structure of what I am trying to do:\nSELECT table1.ID, table1.name, table2.ID, table2.address\nFROM table1, table2\nWHERE table1.ID = table2.ID\nWhat is the basic structure